# Télecharger le jeu de données

In [52]:
from pathlib import Path
import os
# Si dossier /dataset vide ou que son contenu est vide, alors télécharger les données
rootPath = Path(".").parent
datasetFolder = rootPath / "dataset"
if not os.path.exists(datasetFolder):
    os.mkdir(datasetFolder)
if "train.csv" not in  os.listdir(datasetFolder):
    import utilitaire
    utilitaire.download_dataset(datasetFolder)

In [53]:
import polars as pl
import polars.selectors as cs
data = pl.read_csv("dataset/train.csv", try_parse_dates=True)

In [54]:
data.shape

(1200000, 21)

In [55]:
data.null_count()

id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,18705,0,44949,18529,109672,0,358075,74076,0,0,364029,6,137882,1,0,77824,0,0,0,0


In [56]:
df = data.drop_nulls()

In [57]:
df.shape

(384004, 21)

In [58]:
df.describe()

statistic,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
str,f64,f64,str,f64,str,f64,str,str,f64,str,str,f64,f64,f64,f64,str,str,str,str,str,f64
"""count""",384004.0,384004.0,"""384004""",384004.0,"""384004""",384004.0,"""384004""","""384004""",384004.0,"""384004""","""384004""",384004.0,384004.0,384004.0,384004.0,"""384004""","""384004""","""384004""","""384004""","""384004""",384004.0
"""null_count""",0.0,0.0,"""0""",0.0,"""0""",0.0,"""0""","""0""",0.0,"""0""","""0""",0.0,0.0,0.0,0.0,"""0""","""0""","""0""","""0""","""0""",0.0
"""mean""",599960.992656,41.139394,null,33072.900087,null,2.014088,null,null,25.580094,null,null,0.996247,9.561679,594.01282,5.020357,"""2022-02-13 09:25:48.927852""",null,null,null,null,1113.187016
"""std""",346158.471585,13.531632,null,32370.321642,null,1.415478,null,null,12.194086,null,null,0.978161,5.774009,149.607696,2.596525,null,null,null,null,null,868.688359
"""min""",0.0,18.0,"""Female""",2.0,"""Divorced""",0.0,"""Bachelor's""","""Employed""",2.024415,"""Rural""","""Basic""",0.0,0.0,300.0,1.0,"""2019-08-17 15:21:39.080371""","""Average""","""No""","""Daily""","""Apartment""",20.0
"""25%""",300582.0,30.0,null,8069.0,null,1.0,null,null,15.878551,null,null,0.0,5.0,470.0,3.0,"""2020-11-20 15:21:39.168099""",null,null,null,null,519.0
"""50%""",599791.0,41.0,null,24182.0,null,2.0,null,null,24.580045,null,null,1.0,10.0,596.0,5.0,"""2022-02-13 15:21:39.206847""",null,null,null,null,882.0
"""75%""",898899.0,53.0,null,44803.0,null,3.0,null,null,34.486416,null,null,2.0,15.0,722.0,7.0,"""2023-05-07 15:21:39.237118""",null,null,null,null,1520.0
"""max""",1.19999e6,64.0,"""Male""",149997.0,"""Single""",4.0,"""PhD""","""Unemployed""",58.452478,"""Urban""","""Premium""",8.0,19.0,849.0,9.0,"""2024-08-15 15:21:39.287115""","""Poor""","""Yes""","""Weekly""","""House""",4997.0


In [59]:
df =  df.drop(["id", "Policy Start Date", "Credit Score", "Customer Feedback"])

from polars import String
features_modalities = []
for feature_name, feature_type in df.schema.items():
    if isinstance(feature_type, String):
        modalities = df[feature_name].unique().to_list()
        features_modalities.append(f"{feature_name} ({feature_type}) : " + ",".join(modalities) + "\n")
    else:
        features_modalities.append(f"{feature_name} ({feature_type}) \n")

with open("features_modalities.txt", "w") as f:
    f.writelines(features_modalities)

In [60]:
df = df.to_dummies(cs.string(), separator="_",  drop_first=True)

In [61]:
df.describe()

statistic,Age,Gender_Male,Annual Income,Marital Status_Divorced,Marital Status_Single,Number of Dependents,Education Level_High School,Education Level_Master's,Education Level_PhD,Occupation_Employed,Occupation_Unemployed,Health Score,Location_Rural,Location_Suburban,Policy Type_Basic,Policy Type_Comprehensive,Previous Claims,Vehicle Age,Insurance Duration,Smoking Status_Yes,Exercise Frequency_Daily,Exercise Frequency_Monthly,Exercise Frequency_Rarely,Property Type_Apartment,Property Type_Condo,Premium Amount
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",384004.0,384004.0,384004.0,384004.0,384004.0,384004.0,384004.0,384004.0,384004.0,384004.0,384004.0,384004.0,384004.0,384004.0,384004.0,384004.0,384004.0,384004.0,384004.0,384004.0,384004.0,384004.0,384004.0,384004.0,384004.0,384004.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",41.139394,0.502555,33072.900087,0.333562,0.334158,2.014088,0.240461,0.251984,0.253859,0.335085,0.328257,25.580094,0.333174,0.33434,0.332853,0.333017,0.996247,9.561679,5.020357,0.500964,0.245247,0.249779,0.249807,0.3352,0.33253,1113.187016
"""std""",13.531632,0.499994,32370.321642,0.471486,0.471696,1.415478,0.427364,0.434153,0.435219,0.472021,0.46958,12.194086,0.471349,0.47176,0.471235,0.471293,0.978161,5.774009,2.596525,0.5,0.430234,0.432885,0.432902,0.472061,0.47112,868.688359
"""min""",18.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.024415,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0
"""25%""",30.0,0.0,8069.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,15.878551,0.0,0.0,0.0,0.0,0.0,5.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,519.0
"""50%""",41.0,1.0,24182.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,24.580045,0.0,0.0,0.0,0.0,1.0,10.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,882.0
"""75%""",53.0,1.0,44803.0,1.0,1.0,3.0,0.0,1.0,1.0,1.0,1.0,34.486416,1.0,1.0,1.0,1.0,2.0,15.0,7.0,1.0,0.0,0.0,0.0,1.0,1.0,1520.0
"""max""",64.0,1.0,149997.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,58.452478,1.0,1.0,1.0,1.0,8.0,19.0,9.0,1.0,1.0,1.0,1.0,1.0,1.0,4997.0


In [62]:
y = df['Premium Amount']
X = df.drop('Premium Amount')

In [63]:
import sklearn
sklearn.__version__

'1.6.1'

In [64]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor

In [65]:
lin_reg = LinearRegression()
lin_reg.fit(X, y)

LinearRegression()

In [66]:
lin_reg.coef_

array([-3.32047327e-01,  1.89259233e+00, -3.88836593e-04, -2.70095633e+00,
        7.54623788e-01, -4.23428851e-01,  5.75656791e+00, -2.44073627e+00,
       -1.44689715e+00,  4.32082284e+00, -8.81746664e-01,  1.01165864e+00,
       -2.17236236e+00,  2.10305867e+00,  2.54756939e+00,  1.92423476e+00,
        3.92685534e+01,  2.75543067e-01, -1.29790604e-01,  1.23699048e+00,
        2.62279824e+00, -2.49287621e+00, -7.00325307e-01, -8.50210023e-01,
        1.43197597e+00])

In [67]:
gb_model = GradientBoostingRegressor()
import time
start_time = time.time()
gb_model.fit(X, y)
print("--- %s seconds ---" % (time.time() - start_time))


--- 158.59606909751892 seconds ---


In [68]:
kn_model = KNeighborsRegressor()
import time
start_time = time.time()
kn_model.fit(X, y)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.08431267738342285 seconds ---


In [69]:
# Enregistrer les modèles
from pathlib import Path
import os
model_location = Path(".").parent / "models"
if not os.path.exists(model_location):
  os.mkdir(model_location)

import pickle

model_export_names = ["LinearReg_Model.pkl", "GradientB_Model.pkl", "KNN_Model.pkl"]

for model, export_name in zip([lin_reg, gb_model, kn_model], model_export_names):
  with open(model_location / export_name, "wb") as f:
    pickle.dump(model, f)
